In [ ]:
import pandas as pd

In [82]:
from pathlib import Path

base = Path(r"J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements")

year_dirs = [p for p in base.iterdir() if p.is_dir() and p.name.isdigit()]
if not year_dirs:
    raise FileNotFoundError(f"No year folders found under {base!s}")
latest_year_dir = max(year_dirs, key=lambda p: int(p.name))

pattern_files = list(
    latest_year_dir.glob("Cumulative Report - Operating Statements - *.xlsx")
)
xlsx_files = pattern_files or list(latest_year_dir.glob("*.xlsx"))
if not xlsx_files:
    raise FileNotFoundError(f"No .xlsx files found in {latest_year_dir!s}")

latest_report = max(xlsx_files, key=lambda p: p.stat().st_mtime)

report_path = latest_report
print(report_path)

dor = pd.read_excel(report_path, sheet_name="Cumulative Report", skiprows=8)

J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Monthly Operating Statements\2025\Cumulative Report - Operating Statements - 0925 - Hard Coded.xlsx


In [83]:
# drop Unnamed: 0 only if it exists
if "Unnamed: 0" in dor.columns:
    dor = dor.drop(columns=["Unnamed: 0"])

# desired columns (use canonical names)
desired_cols = [
    "Project ID",
    "Project Title",
    "Program Area",
    "Funder Type",
    "Principal Investigator (PI)",
    "Award Term Start Date",
    "Project Status",
    "Total Cash Receipts",
    "Total Personnel",
    "Total Contractual/ Outside Services Costs",
    "Total Non-Personnel",
    "Total Cost",
]

# normalize helper to match columns ignoring whitespace/newlines/case
normalize = lambda s: "".join(s.split()).lower() if isinstance(s, str) else s
col_map = {normalize(c): c for c in dor.columns}

# build selected column list from available columns (skip missing ones)
selected = []
missing = []
for c in desired_cols:
    key = normalize(c)
    if key in col_map:
        selected.append(col_map[key])
    else:
        missing.append(c)

if missing:
    print(
        f"Warning: these desired columns were not found and will be skipped: {missing}"
    )

# subset dataframe to the selected (available) columns
dor = dor[selected]

print(dor.dtypes)

dor.to_excel(
    "C:\\Users\\O304312\\OneDrive - Kaiser Permanente\\Documents\\Tableau Dashboards\\New Financial Snapshot\\Data\\DOR Data Preprocessed.xlsx",
    index=False,
)

Project ID                                            object
Project Title                                         object
Program Area                                          object
Funder Type                                           object
Principal Investigator (PI)                           object
Award Term Start Date                         datetime64[ns]
Project Status                                        object
Total Cash Receipts                                  float64
Total Personnel                                      float64
Total Contractual/\nOutside Services Costs           float64
Total \nNon-Personnel                                float64
Total Cost                                           float64
dtype: object


In [84]:
txn_base = Path(r"J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Transaction Detail")
pattern = "CTP Transaction Detail *.xlsx"
matches = list(txn_base.glob(pattern))

if matches:
    ctp_path = max(matches, key=lambda p: p.stat().st_mtime)
else:

    fallback = txn_base / "CTP Transaction Detail 103125.xlsx"
    if fallback.exists():
        ctp_path = fallback
    else:
        raise FileNotFoundError(
            f"No files matching {pattern!s} and fallback {fallback!s} not found in {txn_base!s}"
        )

print("Loading:", ctp_path)

ctp_hours = pd.read_excel(ctp_path, sheet_name="Hours", header=[8, 9, 10, 11])


def tidy(col):
    parts = [
        str(x).strip()
        for x in col
        if str(x).strip() not in {"nan", ""} and not str(x).startswith("Unnamed")
    ]
    return "_".join(parts).strip("_")


ctp_hours.columns = [tidy(col) for col in ctp_hours.columns]

ctp_hours = ctp_hours.rename(columns={"Project": "Project ID"})

mask = ctp_hours["Project ID"].astype(str).str.strip().str.lower().str.endswith("total")
ctp_hours = ctp_hours[mask].copy()
ctp_hours.reset_index(drop=True, inplace=True)

rng_mask = (
    ctp_hours["Project ID"].astype(str).str.match(r"^(RNG\d+)\s+Total$", na=False)
)
ctp_hours.loc[rng_mask, "Project ID"] = (
    ctp_hours.loc[rng_mask, "Project ID"]
    .astype(str)
    .str.replace(r"^(RNG\d+)\s+Total$", r"\1", regex=True)
    .str.strip()
)


print("Columns:", ctp_hours.columns.tolist())

print("Loaded dataframe shape:", ctp_hours.shape)

ctp_hours.to_excel(
    "C:\\Users\\O304312\\OneDrive - Kaiser Permanente\\Documents\\Tableau Dashboards\\New Financial Snapshot\\Data\\DOR Personnel.xlsx",
    index=False,
)

Loading: J:\ADMIN-eFILES\CHEN_W154867_VXC\z_Reports\Transaction Detail\CTP Transaction Detail 103125.xlsx
Columns: ['Project ID', 'NUID', 'Name', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025_Qtr1_Jan', '2025_Qtr1_Feb', '2025_Qtr1_Mar', '2025_Qtr2_Apr', '2025_Qtr2_May', '2025_Qtr2_Jun', '2025_Qtr3_Jul', '2025_Qtr3_Aug', '2025_Qtr3_Sep', '2025_Qtr4_Oct', '2025 Total', 'Grand Total']
Loaded dataframe shape: (214, 26)


In [ ]:
import re
import json

vp_folder = Path("J:\\VIEWPOINT\\SiteStudyDetails_Response")

pattern = re.compile(r"^\d{4}-\d{2}-\d{2}-SiteStudyDetails\.json$")

matches = [p for p in vp_folder.iterdir() if p.is_file() and pattern.match(p.name)]
if not matches:
    raise FileNotFoundError(f"No SiteStudyDetails json files found in {vp_folder!s}")

latest_json = max(matches, key=lambda p: p.stat().st_mtime)
print("Loading:", latest_json)

with latest_json.open("r", encoding="utf-8") as f:
    site_details = json.load(f)

try:
    ss_df = pd.json_normalize(site_details)
    print("Converted to DataFrame with shape:", ss_df.shape)
except Exception:
    ss_df = None
    print("JSON loaded into 'site_details' (not converted to DataFrame).")

Loading: J:\VIEWPOINT\SiteStudyDetails_Response\2025-12-01-SiteStudyDetails.json
Converted to DataFrame with shape: (1074, 40)


In [ ]:
viewpoint = ss_df.copy(deep=True)

dor["ID"] = dor["Project Title"]

program_col = next(
    (c for c in dor.columns if normalize(c) == normalize("Program Area")), None
)

if program_col is not None:

    def _remove_prog(id_val, prog_val):
        if pd.isna(id_val) or pd.isna(prog_val):
            return id_val
        id_s = str(id_val)
        prog_s = str(prog_val).strip()
        if not prog_s:
            return id_s
        out = re.sub(re.escape(prog_s), "", id_s, flags=re.IGNORECASE)
        out = re.sub(r"[\-\–\—:;\/]+", " ", out)
        out = " ".join(out.split()).strip()
        return out

    dor["ID"] = dor.apply(lambda r: _remove_prog(r["ID"], r[program_col]), axis=1)
else:
    print("Warning: 'Program Area' column not found in dor; ID left unchanged.")

In [ ]:
viewpoint["site_study_id"] = viewpoint["site_study_id"].astype(str)
viewpoint["site_study_id"] = (
    viewpoint["site_study_id"].str.split("(").str[0].str.strip()
)

viewpoint["site_study_id"] = viewpoint["site_study_id"].str.replace(
    r"#", "", regex=True
)
viewpoint["site_study_id"] = viewpoint["site_study_id"].str.replace(
    r"-", " ", regex=True
)
# viewpoint["site_study_id"] = viewpoint["site_study_id"].str.replace(r" ", "", regex=True)

In [ ]:
# mask = dor["ID"].notna()
# dor.loc[mask, "ID"] = (
#     dor.loc[mask, "ID"].astype(str)
#     .str.strip()
#     .str.replace(r"^\S+\s*", "", regex=True)
# )

# dor["ID"] = dor["ID"].str.replace(r"-", "", regex=True)
# dor["ID"] = dor["ID"].str.replace(r" ", "", regex=True)

In [89]:
from IPython.display import display, HTML
import html as _html

left_list = [str(x).strip() for x in dor["ID"].tolist()]
right_list = [str(x).strip() for x in viewpoint["site_study_id"].tolist()]

left_sorted = sorted(left_list, key=lambda s: s.casefold())
right_sorted = sorted(right_list, key=lambda s: s.casefold())

left = "\n".join(left_sorted)
right = "\n".join(right_sorted)

html_content = f"""
<div style="display:flex;gap:12px;">
    <div style="flex:1;max-height:400px;overflow:auto;border:1px solid #ccc;padding:8px;">
        <strong>DOR ID (all, sorted)</strong>
        <pre style="white-space:pre-wrap;">{_html.escape(left)}</pre>
    </div>
    <div style="flex:1;max-height:400px;overflow:auto;border:1px solid #ccc;padding:8px;">
        <strong>Viewpoint site_study_id (all, sorted)</strong>
        <pre style="white-space:pre-wrap;">{_html.escape(right)}</pre>
    </div>
</div>
"""

display(HTML(html_content))